<a href="https://colab.research.google.com/github/nattaran/health-tequity-case-nasrin/blob/main/VoicePipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print ("Hello Word")

Hello Word


In [ ]:
import os

# Update this path to match your Google Drive folder


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/health-tequity-case'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd

'/content'

In [ ]:
cd ..
pwd

SyntaxError: invalid syntax (ipython-input-2078655017.py, line 1)

In [ ]:
cd ..

/


In [ ]:
pwd

'/'

In [ ]:
ls

bin@                        lib32@                    root/
boot/                       lib64@                    run/
content/                    libx32@                   sbin@
cuda-keyring_1.1-1_all.deb  media/                    srv/
datalab/                    mnt/                      sys/
dev/                        NGC-DL-CONTAINER-LICENSE  tmp/
etc/                        opt/                      tools/
home/                       proc/                     usr/
kaggle/                     python-apt/               var/
lib@                        python-apt.tar.xz*


In [ ]:
mkdir health_tequaity_assessment


In [ ]:
ls

bin@                         home/    NGC-DL-CONTAINER-LICENSE  srv/
boot/                        kaggle/  opt/                      sys/
content/                     lib@     proc/                     tmp/
cuda-keyring_1.1-1_all.deb   lib32@   python-apt/               tools/
datalab/                     lib64@   python-apt.tar.xz*        usr/
dev/                         libx32@  root/                     var/
etc/                         media/   run/
health_tequaity_assessment/  mnt/     sbin@


In [ ]:
cd health_tequaity_assessment/


/health_tequaity_assessment


In [ ]:
ls